In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

## 추천 사이트 크롤링

In [2]:
# G마켓 슈퍼딜 사이트
url = 'http://corners.gmarket.co.kr/SuperDeals/' # 추천 카테고리

# 웹 서버 요청
resp = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(resp.text, 'html.parser')

In [3]:
item_list_tag = soup.find('ul', attrs={'class':'item_list'})
item_list_tags = item_list_tag.find_all('li')

title_list = []
sale_list = []
price_list = []
buy_list = []

for li_tag in item_list_tags:
    title_text = li_tag.find('span', attrs = {'class': 'title'}).get_text()
    title_list.append(title_text)
    
    sale_text = li_tag.find('em', attrs = {'class' : 'sale'})
    if sale_text is None:
        sale_list.append('0')
    else:
        sale_list.append(sale_text.get_text().strip('% 할인'))

    price_text = li_tag.find('span', attrs = {'class': 'price'}).find('strong').get_text().strip('원') # rent-per 부분 처리를 어떻게 할까?
    price_list.append(price_text)

    buy_text = li_tag.find('span', attrs = {'class':'buy'}).find('strong').get_text()
    buy_list.append(buy_text)


print(title_list)
print(sale_list)
print(price_list)
print(buy_list)

['15%쿠폰+카드10 탑텐 본사 쿨나시/반팔티/원피스 외', '[15%/3개]1+1+1 UV 수분/매트 선크림 자외선차단+증정', '15%+크록스 본사 봄맞이 키즈 인기상품 모음', '[15+5%쿠폰] 홈쇼핑 티셔츠 7종세트 장당2,800원?', '쿤달 네이처 샴푸 베이비파우더 3개+사은품', '얼티브 오리지널 190ML x24입 +[증정] 비건커피 4개', '웰치스 제로 포도 355ml 24캔X2개(총48캔)', '상하 유기농 치즈 1~4단계 60매', '독일군 스니커즈 외 데일리 슈즈 추천', 'LG정수기+상품권30만+6천만경품+결합시 3천원할인!', '보람상조+LG삼성최신가전 지원+만기전액환급+상품권증정', '[쿠폰가 41,900원] 이마트인기 이맛쌀 20kg', '[10%] 스파클생수 2Lx30병', '[15%] 종근당건강 락토핏 골드 보장균수 2배 50포 3통(150포)', '어가찬 바다일품 백 진미채 / 홍 진미채1kg ', '[10%]14브릭스 천혜향 4kg 중대과', '[15%쿠폰]국산 10켤레 남여 기본 무지 발목양말', '리챔 오리지널 200g 10캔', '[15+10%] 더블 헤어 새치마커 30초 빠른커버/ 2개 추가할인 #7820', '환절기 필수!! 엘빈즈 어린이 배도라지 40팩', '닭다리살로 만든 너겟 1kg+1kg', '최상급 통통 왕새우 1kg 40-45미 / 신선급냉', '본사직영/폴햄키즈 간편한 외출에 딱! 데일리템', '[15%] 레쓰비 마일드 캔커피 160ml x 60캔', '모나리자 보타닉 블라썸(27m30롤)x3개', '[당일배송] 믿고먹는 안심 농협쌀 20KG', '[15%+10%]LPP트리트먼트 헤어팩 1000ml 1+1 +증정', '서울멸균우유 200ml 24입+24입 (2박스)', '[15%+10%] 치간칫솔 I형 SSSS(0.4mm) 50P x 2팩', '바브키즈 봄 신상 업데이트 완료! 상하복/맨투맨/팬츠', '2023 NEW 네이처메이드 밴드/팬티 1~7단계 모음전', '(선물하기 특가) S-OIL 모바일

In [4]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가
dict_data = {
    'title' : title_list,
    'sale' : sale_list,
    'price' : price_list,
    'buy' : buy_list
}

# 판다스 데이터프레임으로 변환
df_recom = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_recom

,title,sale,price,buy
0,15%쿠폰+카드10 탑텐 본사 쿨나시/반팔티/원피스 외,53,"9,290",4만+
1,[15%/3개]1+1+1 UV 수분/매트 선크림 자외선차단+증정,70,"12,900","2,046"
2,15%+크록스 본사 봄맞이 키즈 인기상품 모음,63,"25,760","1,032"
3,"[15+5%쿠폰] 홈쇼핑 티셔츠 7종세트 장당2,800원?",69,"24,800",805
4,쿤달 네이처 샴푸 베이비파우더 3개+사은품,0,"32,700","2,805"
...,...,...,...,...
289,괌 PIC 골드카드 4일/5일 (서커스+소아동반룸UP),4,"764,000",7
290,제주도 실시간 편도 항공권,0,"120,600",NEW
291,순천 정원박람회 가든스테이 쉴랑게,0,"450,000",3
292,[직구특가] 아미/메종키츠네/프라다 외 명품 BEST,18,"184,500",NEW


## 카테고리별로 크롤링

In [5]:
def category_url(num,cate):
    url = 'http://corners.gmarket.co.kr/SuperDeals/?DispType=1&GroupNo=400000{}&SubNo=0&CateIndex=0'.format(num)
    
    # 웹 서버 요청
    resp = requests.get(url)

    # HTML 파싱
    soup = BeautifulSoup(resp.text, 'html.parser')

    item_list_tag = soup.find('ul', attrs={'class':'item_list'})
    item_list_tags = item_list_tag.find_all('li')

    title_list = []
    sale_list = []
    price_list = []
    buy_list = []

    for li_tag in item_list_tags:
        title_text = li_tag.find('span', attrs = {'class': 'title'}).get_text()
        title_list.append(title_text)
    
        sale_text = li_tag.find('em', attrs = {'class' : 'sale'})
        if sale_text is None:
            sale_list.append('0')
        else:
            sale_list.append(sale_text.get_text().strip('% 할인'))

        price_text = li_tag.find('span', attrs = {'class': 'price'}).find('strong').get_text().strip('원') # rent-per 부분 처리를 어떻게 할까?
        price_list.append(price_text)

        buy_text = li_tag.find('span', attrs = {'class':'buy'}).find('strong').get_text()
        buy_list.append(buy_text)

    # 딕셔너리 형식으로 항목별 리스트를 원소로 추가
    dict_data = {
        'title' : title_list,
        'sale' : sale_list,
        'price' : price_list,
        'buy' : buy_list
    }

    # 판다스 데이터프레임으로 변환
    df_data = pd.DataFrame(dict_data)
    df_data['category'] = cate

    # 변환 결과를 확인
    return df_data

In [6]:
category = {
    '브랜드패션'  : 135,
    '트랜드패션'  : 136,
    '뷰티·잡화'  : 137,
    '유아동'      : 138,
    '식품'        : 139,
    '생필품'      : 140,
    '가구·침구'  : 141,
    '생활·건강'  : 143,
    '스포츠·레저': 146,
    '가전·컴퓨터': 148,
    '디지털'      : 149, 
    '문구·도서'  : 151,
    '여행'        : 152,
    'e쿠폰'       : 153,
    '백화점'      : 164
}

In [7]:
df_cate = pd.DataFrame()
for key in category:
    cate = key
    value = category.get(cate)
    df_cate = pd.concat([df_cate, category_url(value, cate)], ignore_index = True)

df_cate

,title,sale,price,buy,category
0,15%쿠폰+카드10 탑텐 본사 쿨나시/반팔티/원피스 외,53,"9,290",4만+,브랜드패션
1,[15+10%] 올리비아로렌 봄맞이 코디제안,69,"22,000",3만+,브랜드패션
2,[15%]닉스 봄/간절기 세일 코트/점퍼/자켓/팬츠 ~80%,70,"29,400",10만+,브랜드패션
3,울시 남여 골프 카라티 초특가전,4,"14,160",390,브랜드패션
4,15%+17% 에스콰이아 봄맞이 남여 슈즈 잡화 모음전,70,"48,300",5만+,브랜드패션
...,...,...,...,...,...
1756,클라이드 공용 스누피 스타디움 자켓 외,20,"35,480",179,백화점
1757,푸마 스피드퓨전 SPARCO/SPEEDFUSION x Sparco,21,"51,350",132,백화점
1758,[레노마_셔츠(남성)] 2023 레노마 스판 드레스셔츠 슬림/일반핏 특집전,0,"35,000","1,089",백화점
1759,블랙야크키즈 BKO카메론맨투맨세트,22,"61,220",272,백화점
